In [ ]:
pip install d20

In [1]:
import pandas as pd
import random
from d20 import roll

In [2]:
excel_file = 'https://github.com/btaylor77/dnd/blob/main/beyond_pandas.xlsx?raw=true'
#excel_file = '/Users/Brad/Documents/git/dnd/beyond_pandas.xlsx'
hoard_df = pd.read_excel(excel_file,'hoard_items',engine='openpyxl').fillna(0)
spell_df = pd.read_excel(excel_file,'spell_list',index_col='Spell Name',engine='openpyxl')
class_df = pd.read_excel(excel_file,'class_spell_list',index_col='Spell Name',engine='openpyxl')
items_df = pd.read_excel(excel_file,'magic_items',index_col='item_name',engine='openpyxl')
coins_df = pd.read_excel(excel_file,'coins',engine='openpyxl').fillna(0)
treasure_df = pd.read_excel(excel_file,'treasure',engine='openpyxl').fillna(0)
damage_types = ['acid','cold','fire','force','lightning','necrotic','psychic','poison','radiant','thunder']
monster_types = ['Abberation','Beast','Celestial','Construct','Dragon','Elemental','Fey','Fiend','Giant','Humanoid','Monstrosity','Ooze','Plant','Undead']
swords = ['Greatsword','Longsword','Rapier','Scimitar','Shortsword']
weapons = swords + ['Battleaxe','Blowgun','Club','Crossbow, hand','Crossbow, heavy','Crossbow, light','Dagger','Dart','Flail','Glaive','Greataxe','Greatclub','Halberd','Handaxe','Javelin','Lance','Light hammer','Longbow','Mace','Maul','Morningstar','Net','Pike','Quarterstaff','Shortbow','Sickle','Sling','Spear','Trident','War pick','Warhammer','Whip',]

party = [('bard','base'),('bard','optional'),
          ('cleric','peace'),('cleric','base'),('cleric','optional'),
          ('warlock','base'),('warlock','fiend'),('warlock','optional'),
          ('wizard','base'),('wizard','optional')
        ]
all_class = [('artificer','base'),('bard','base'),('cleric','base'),('druid','base'),('paladin','base'),('ranger','base'),('sorcerer','base'),('wizard','base'),('warlock','base')]
arcane_class = [('artificer','base'),('bard','base'),('sorcerer','base'),('wizard','base'),('warlock','base')]
divine_class = [('cleric','base'),('druid','base'),('paladin','base'),('ranger','base')]


player_sb = ["Acid Splash","Animate Objects","Banishment","Bigby's Hand","Blight","Color Spray",
             "Comprehend Languages","Cone of Cold","Continual Flame","Counterspell","Detect Magic",
             "Dispel Magic","Earth Tremor","Earthbind","Far Step","Find Familiar","Fly","Fog Cloud",
             "Greater Invisibility","Ice Knife","Identify","Leomund’s Tiny Hut","Lightning Bolt",
             "Mage Armor","Mage Hand","Magic Missile","Mind Sliver","Mirror Image","Misty Step",
             "Poison Spray","Polymorph","Ray of Frost","Shield","Sleep","Snilloc’s Snowball Swarm",
             "Steel Wind Strike","Stoneskin","Summon Fey","Tasha's Mind Whip","Vitriolic Sphere",
             "Water Breathing","Unseen Servant","Skywrite","Shatter","Alarm","Tenser's Floating Disk",
             "Haste","Protection from Energy"]

school_weight = {
    'abjuration':2,
    'conjuration':3,
    'divination':2,
    'enchantment':1,
    'evocation':1,
    'illusion':1,
    'necromancy':10,
    'transmutation':1,
}
spellbook = class_df[class_df['Class']=='wizard'].copy()
spellbook['weight'] = spellbook.apply(lambda x: school_weight[x['School']], axis=1)
spellbook.loc[spellbook.index.isin(player_sb), 'weight']= 0.5
spellbook['weight_factor'] = spellbook.apply(lambda x: random.random() / x['weight'],axis=1)

spellbook = spellbook.sort_values('weight_factor')

In [8]:
def roll_spells(min_level,max_level,classes,n=1):
    target_df = class_df.copy()
    target_df = target_df[(target_df['Level'] >= min_level) & (target_df['Level'] <= max_level) ]
    target_df = target_df[target_df[["Class","Subclass"]].apply(tuple, 1).isin(classes)]
    spells = target_df.index.values.tolist()
    select = pd.Series(random.choices(spells,k=n)).unique()
    target_df = spell_df.loc[select].sort_values(['Level','Spell Name'])
    return target_df

def roll_magic_items(table,n):
    if n > 0:
        target_df = items_df.copy()
        target_df = target_df[target_df['table_name']==table]
        target_df = target_df.reset_index()
        try:
            select = random.choices(target_df.index.values.tolist(),k=n,weights=list(target_df['weight']))
        except:
            print(table)
        target_df = target_df.loc[select]
        target_df['item_name'] = target_df.apply(lambda row: 'Scroll Of ' + roll_spells(row['spell_level'],row['spell_level'],party,n=1).index.values[0] if row['spell_level'] in range(9) else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(damage_types) + ')' if row['damage_type_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(monster_types) + ')' if row['monster_type_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(swords) + ')' if row['sword_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(weapons) + ')' if row['weapon_flag'] == 1 else row['item_name'], axis=1)
        target_df = target_df.sort_values('item_name')
    else:
        target_df = pd.DataFrame(columns=['item_name'])
    return target_df

def roll_coins(crs=[],coin_type='single'):
    result_df = pd.DataFrame()
    for cr in crs:
        target_df = coins_df.copy()
        target_df = target_df[(target_df['min_cr']<= cr) & (target_df['max_cr'] >= cr) & (target_df['coin_type']==coin_type)]
        select = random.choices(target_df.index.values.tolist(),k=1,weights=list(target_df['weight']))
        target_df = target_df.loc[select][['CP','SP','EP','GP','PP']]
        result_df = result_df.append(target_df)
        result_df['CP'] = result_df['CP'].apply(lambda x: roll(str(x)).total)
        result_df['SP'] = result_df['SP'].apply(lambda x: roll(str(x)).total)
        result_df['EP'] = result_df['EP'].apply(lambda x: roll(str(x)).total)
        result_df['GP'] = result_df['GP'].apply(lambda x: roll(str(x)).total)
        result_df['PP'] = result_df['PP'].apply(lambda x: roll(str(x)).total)
        total_coins = pd.DataFrame(result_df.sum(axis=0)).transpose()
#        result_df = result_df.apply(lambda x: roll(x).total)
    result_df = total_coins.transpose().reset_index()
    result_df['treasure'] = result_df.apply(lambda x: str(x[0]) + ' ' + x['index'].lower(),axis=1)
    coins = result_df[result_df[0] > 0]['treasure'].values.tolist()
    return coins

def roll_treasure(gp,n):
    treasure = []
    if n > 0:
        target_df = treasure_df[treasure_df['GP']==gp]
        treasure = []
        for i in range(n):
            select = random.choice(target_df.index.values.tolist())
            treasure.append(select)
        target_df = target_df.loc[treasure].groupby('treasure_name').count()
        target_df = target_df.rename(columns={"GP":"count"})
        target_df['each'] = target_df.apply(lambda row: ' each' if row['count']>1 else '',axis=1)

        target_df = target_df.reset_index()
        target_df['treasure_desc'] = target_df.apply(lambda row: str(row['count']) + ' ' + row['treasure_name'] + ' worth ' + str(gp) + 'gp' + row['each'],axis=1)
        treasure = target_df['treasure_desc'].values.tolist()
    return treasure

def roll_hoard(cr):
    loot = []
    target_df = hoard_df.copy()
    target_df['cr'] = cr
    target_df = target_df[(target_df['min_cr']<= cr) & (target_df['max_cr'] >= cr)]
    select = random.choices(target_df.index.values.tolist(),k=1,weights=list(target_df['weight']))
    target_df = target_df.loc[select]
    target_df['treasure_die'] = target_df['treasure_die'].apply(lambda x: roll(str(x)).total)
    target_df['magic_item_die_1'] = target_df['magic_item_die_1'].apply(lambda x: roll(str(x)).total)
    target_df['magic_item_die_2'] = target_df['magic_item_die_2'].apply(lambda x: roll(str(x)).total)
    target_df = target_df.drop(columns=['min_cr','max_cr','weight'])
    items1 = list(roll_magic_items(target_df['magic_item_table_1'].values[0],target_df['magic_item_die_1'].values[0])['item_name'].values.tolist())
    items2 = list(roll_magic_items(target_df['magic_item_table_2'].values[0],target_df['magic_item_die_2'].values[0])['item_name'].values.tolist())
    treasure = roll_treasure(target_df['treasure_value'].values[0],target_df['treasure_die'].values[0])
    coins = roll_coins([cr],'hoard')
    loot = loot + treasure + coins + items1 + items2
    target_df = target_df.reset_index(drop=True)

    return loot#target_df

def generate_spellbook(min_level=1,max_level=9,n=6,weight={}):
    select = []
    df = spellbook.copy()

    for i in range(min_level,max_level+1):
        lvl_df = df[df['Level']==i]
        lvl_df = lvl_df.sort_values('weight_factor')
#        lvl_spells = lvl_spells.loc[lvl_spells.index.repeat(lvl_spells.weight)].reset_index(drop=False) # rows duplicated based on weight
        if n > 0:
            if i < max_level:
                x = max(1,random.randrange((n+1)//4,(n+1)//2))
            else:
                x = n
            lvl_select = lvl_df.sample(frac=1).head(x).index.values.tolist()
            #lvl_select = random.choices(lvl_df.index.values.tolist(),k=x,weights=list(lvl_df['weight']))
            select = select + lvl_select
            n = n - x
    df = df.loc[select].sort_values(['Level','School'])
    return df

def display_spellbook(df):
    for level in df['Level'].unique():
        lvldf = df[df['Level']==level].sort_index()
        print(f'Level {level} Spells:' + '(' + str(len(lvldf)) + ')')
        print(", ".join(map(str,lvldf.index.values)))
        print('')


In [24]:
for i in roll_hoard(17): #+ roll_hoard(15):
    print(i)

2 Embroidered glove set with jewel chips worth 2500gp each
2 Eye patch with a mock eye set in blue sapphire and moonstone worth 2500gp each
1 Jeweled anklet worth 2500gp
1 Old masterpiece painting worth 2500gp
3 Platinum bracelet set with a sapphire worth 2500gp each
42000 gp
25000 pp
Bag of Beans
Potion of Healing x2
Potion of Healing x2
Potion of Mind Reading
Potion of Stone Giant Strength
Potion of Stone Giant Strength
Scroll of Protection (Monstrosity)


In [25]:
for i in list(roll_magic_items('H',6)['item_name'].values.tolist()):
    print(i)

Arcane Grimoire, +3
Carpet of Flying
Cauldron of Rebirth
Dwarven Plate
Rod of the Pact Keeper, +3
Tome of Understanding


In [26]:
total = []
total = total + roll_hoard(15)
total = total + roll_hoard(17)
total

['9000 gp',
 '1800 pp',
 '3 Blue Sapphire worth 1000gp each',
 '2 Emerald worth 1000gp each',
 '2 Fire Opal worth 1000gp each',
 '2 Star Sapphire worth 1000gp each',
 '43000 gp',
 '30000 pp',
 'Armor, +2 Half Plate',
 'Armor, +2 Studded Leather',
 'Defender (Greatsword)',
 'Teeth of Dahlver-Nar']

In [27]:
for i in total:
    print(i)

9000 gp
1800 pp
3 Blue Sapphire worth 1000gp each
2 Emerald worth 1000gp each
2 Fire Opal worth 1000gp each
2 Star Sapphire worth 1000gp each
43000 gp
30000 pp
Armor, +2 Half Plate
Armor, +2 Studded Leather
Defender (Greatsword)
Teeth of Dahlver-Nar


In [57]:
for i in roll_coins([4,4,4,4,4,4,4,3,3,5,9],'single'):
    print(i)

62 cp
20 sp
10 ep
246 gp


def generate_spellbook(min_level=1,max_level=9,n=6,weight={}):
    select = []
    df = spellbook.copy()
    for i in range(min_level,max_level+1):
        if n > 0:
            if i < max_level:
                x = max(1,random.randrange((n+1)//4,(n+1)//2))
            else:
                x = n
            lvl_df = df[df['Level']==i]
            lvl_select = random.choices(lvl_df.index.values.tolist(),k=x,weights=list(lvl_df['weight']))
            select = select + lvl_select
            n = n - x
    df = df.loc[pd.Series(select).unique()].sort_values(['Level','School'])
    return df

def display_spellbook(df):
    for level in df['Level'].unique():
        lvldf = df[df['Level']==level].sort_index()
        print(f'Level {level} Spells:' + '(' + str(len(lvldf)) + ')')
        print(", ".join(map(str,lvldf.index.values)))
        print('')


In [ ]:
sb = generate_spellbook(1,6,25,school_weight)

display_spellbook(sb)
len(sb)

In [ ]:
t1 = class_df[class_df['Class']=='wizard'].copy()
t1['weight'] = t1.apply(lambda x: school_weight[x['School']], axis=1)
t1[t1.index.isin(player_sb)]['weight'] = 0.25

In [ ]:
t1.loc[t1.index.isin(player_sb), 'weight']= 0.25

In [ ]:
sb = generate_spellbook(1,6,100,school_weight)

display_spellbook(sb)
len(sb)

In [39]:
sb = generate_spellbook(1,6,50,school_weight)

display_spellbook(sb)
len(sb)

Level 1 Spells:(22)
Alarm, Burning Hands, Catapult, Chromatic Orb, Expeditious Retreat, Feather Fall, Find Familiar, Fog Cloud, Grease, Identify, Longstrider, Mage Armor, Magic Missile, Ray of Sickness, Silent Image, Sleep, Snare, Tasha's Hideous Laughter, Tenser's Floating Disk, Thunderwave, Unseen Servant, Witch Bolt

Level 2 Spells:(7)
Blindness/Deafness, Blur, Continual Flame, Dragon's Breath, Locate Object, Rime's Binding Ice, Shatter

Level 3 Spells:(8)
Clairvoyance, Flame Arrows, Haste, Magic Circle, Nondetection, Summon Fey, Summon Lesser Demons, Thunder Step

Level 4 Spells:(4)
Ice Storm, Leomund's Secret Chest, Locate Creature, Summon Construct

Level 5 Spells:(4)
Danse Macabre, Dawn, Scrying, Steel Wind Strike

Level 6 Spells:(5)
Chain Lightning, Flesh To Stone, Guards And Wards, Investiture of Flame, Move Earth



50

In [26]:
mortus = ["Detect Magic","Mage Armor","Magic Missile","Shield","Mirror Image","Misty Step","Ray of Enfeeblement",
         "Counterspell","Fly","Lightning Bolt","Summon Undead","Vampiric Touch","Banishment","Fire Shield","Blight",
          "Cone of Cold","Wall of Force","Negative Energy Flood","Circle of Death","Finger of Death","Mind Blank",
         "Time Stop"]
mortus_book = spellbook[spellbook.index.isin(mortus)].sort_values('Level')
mortus_book = mortus_book.append(generate_spellbook(6,9,10))

In [27]:
display_spellbook(mortus_book.sort_values('Level'))

Level 1 Spells:(4)
Detect Magic, Mage Armor, Magic Missile, Shield

Level 2 Spells:(3)
Mirror Image, Misty Step, Ray of Enfeeblement

Level 3 Spells:(5)
Counterspell, Fly, Lightning Bolt, Summon Undead, Vampiric Touch

Level 4 Spells:(3)
Banishment, Blight, Fire Shield

Level 5 Spells:(3)
Cone of Cold, Negative Energy Flood, Wall of Force

Level 6 Spells:(3)
Circle of Death, Magic Jar, Tenser's Transformation

Level 7 Spells:(3)
Crown of Stars, Finger of Death, Simulacrum

Level 8 Spells:(2)
Control Weather, Mind Blank

Level 9 Spells:(6)
Astral Projection, Blade of Disaster, Mass Polymorph, Power Word Kill, Prismatic Wall, Time Stop



In [23]:
generate_spellbook(6,9,10)

,Class,Subclass,Level,School,weight,weight_factor
Spell Name,,,,,,
Drawmij's Instant Summons,wizard,base,6,conjuration,3.0,0.270877
Flesh To Stone,wizard,base,6,transmutation,1.0,0.501350
Symbol,wizard,base,7,abjuration,2.0,0.010010
Mirage Arcane,wizard,base,7,illusion,1.0,0.128481
Sunburst,wizard,base,8,evocation,1.0,0.833089
Maddening Darkness,wizard,base,8,evocation,1.0,0.320705
Imprisonment,wizard,base,9,abjuration,2.0,0.260886
Meteor Swarm,wizard,base,9,evocation,1.0,0.596298
Astral Projection,wizard,base,9,necromancy,10.0,0.075082


In [59]:
mortus = ['Detect Magic','Mage Armor','Magic Missile','Shield',
'Mirror Image','Misty Step','Ray of Enfeeblement',
'Counterspell','Fly','Lightning Bolt','Summon Undead','Vampiric Touch',
'Banishment','Blight','Fire Shield',
'Cone of Cold','Negative Energy Flood','Wall of Force',
'Circle of Death','Magic Jar',"Tenser's Transformation",
'Crown of Stars','Finger of Death','Simulacrum',
'Control Weather','Mind Blank',
'Astral Projection','Blade of Disaster','Mass Polymorph','Power Word Kill','Prismatic Wall','Time Stop']

In [70]:
naiell_spells = spellbook[spellbook.index.isin(naiell)].sort_index().merge(spell_df,how = 'inner', left_index=True,right_on = 'Spell Name')
naiell_spells[naiell_spells['Ritual']==1]

,Class,Subclass,Level_x,School_x,weight,weight_factor,Level_y,Concentration,Ritual,School_y,Components,Casting Time,Duration,Range,Tag,Save,Link
Spell Name,,,,,,,,,,,,,,,,,
Alarm,wizard,base,1,abjuration,0.5,1.081578,1st,NaN,1.0,Abjuration,"V, S, M",1 Minute,8 Hours,30 ft,Detection,NaN,"Alarm Ritual Abjuration • V, S, M"
Comprehend Languages,wizard,base,1,divination,0.5,1.777255,1st,NaN,1.0,Divination,"V, S, M",1 Action,1 Hour,Self,Social,NaN,"Comprehend Languages Ritual Divination • V, S, M"
Detect Magic,wizard,base,1,divination,0.5,1.949454,1st,1.0,1.0,Divination,"V, S",1 Action,10 Minutes,Self,Detection,NaN,Detect Magic Concentration Ritual Divination •...
Find Familiar,wizard,base,1,conjuration,0.5,0.679844,1st,NaN,1.0,Conjuration,"V, S, M",1 Hour,Instantaneous,10 ft,Summoning,NaN,"Find Familiar Ritual Conjuration • V, S, M"
Identify,wizard,base,1,divination,0.5,1.033055,1st,NaN,1.0,Divination,"V, S, M",1 Minute,Instantaneous,Touch,Detection,NaN,"Identify Ritual Divination • V, S, M"
Skywrite,wizard,base,2,transmutation,0.5,1.123959,2nd,1.0,1.0,Transmutation,"V, S",1 Action,1 Hour,Sight,Communication (...),NaN,Skywrite Concentration Ritual Transmutation • ...
Unseen Servant,wizard,base,1,conjuration,0.5,1.087406,1st,NaN,1.0,Conjuration,"V, S, M",1 Action,1 Hour,60 ft,Control,NaN,"Unseen Servant Ritual Conjuration • V, S, M"
Water Breathing,wizard,base,3,transmutation,0.5,1.460500,3rd,NaN,1.0,Transmutation,"V, S, M",1 Action,24 Hours,30 ft,Buff,NaN,"Water Breathing Ritual Transmutation • V, S, M"


In [67]:
naiell = ["Acid Splash",
"Alarm",
"Animate Objects",
"Banishment",
"Bigby's Hand",
"Blight",
"Chain Lightning",
"Color Spray",
"Comprehend Languages",
"Cone of Cold",
"Continual Flame",
"Counterspell",
"Detect Magic",
"Dispel Magic",
"Earth Tremor",
"Earthbind",
"Far Step",
"Find Familiar",
"Floating Disk",
"Fly",
"Fog Cloud",
"Globe of Invulnerability",
"Greater Invisibility",
"Haste",
"Ice Knife",
"Identify",
"Leomund’s Tiny Hut",
"Lightning Bolt",
"Mage Armor",
"Magic Missile",
"Mind Sliver",
"Mirror Image",
"Misty Step",
"Poison Spray",
"Polymorph",
"Protection from Energy",
"Ray of Frost",
"Shatter",
"Shield",
"Shocking Grasp",
"Skywrite",
"Sleep",
"Snilloc’s Snowball Swarm",
"Steel Wind Strike",
"Stoneskin",
"Summon Fey",
"Tasha's Mind Whip",
"Unseen Servant",
"Vitriolic Sphere",
"Water Breathing"]